In [1]:
from app.utilities.s3 import list_files, move_file
from app.utilities.database import get_job_status
from app.utilities.rabbitmq import *
from app.utilities.logging import logger

In [2]:
get_job_status('validation/in-progress/1-20250905051437-test12.csv')

'file_queued'

In [3]:
list_files('validation/queued/')

[{'Key': 'validation/queued/1-20250905051437-test12.csv',
  'LastModified': datetime.datetime(2025, 9, 5, 5, 14, 48, 73000, tzinfo=tzutc()),
  'ETag': '"70ce360f86b41e4254b7de1a70b7652d"',
  'Size': 73,
  'StorageClass': 'STANDARD'},
 {'Key': 'validation/queued/1-20250907191002-test13.csv',
  'LastModified': datetime.datetime(2025, 9, 7, 19, 10, 32, 439000, tzinfo=tzutc()),
  'ETag': '"bfec12d988cfaee58ba1a8af52c73025"',
  'Size': 88,
  'StorageClass': 'STANDARD'},
 {'Key': 'validation/queued/1-20250909223541-test14.csv',
  'LastModified': datetime.datetime(2025, 9, 10, 4, 0, 0, 109000, tzinfo=tzutc()),
  'ETag': '"650e5d0f6a3093497aeee4a88f8c4c42"',
  'Size': 145,
  'StorageClass': 'STANDARD'}]

In [4]:
logger.info("Prototypes ran successfully!")

2025-09-10 02:07:36,360 - INFO - Prototypes ran successfully!


In [5]:
queue_agent = QueueAgent(rabbitmq_vhost='mls.batch.pending_validation')

2025-09-10 02:07:37,156 - DEBUG - Connected to RabbitMQ at rabbitmq.apps.cansin.net:5672


In [6]:
queue_agent.create_queue('test-queue-123')

2025-09-10 02:07:37,383 - DEBUG - Created queue: 'test-queue-123'.


True

In [7]:
queues = queue_agent.list_all_queues()
queues

['test-queue-123']

In [8]:
for i in range(3):
    queue_agent.publish_message(queues[0], {'test': f'message{i+1}'})

2025-09-10 02:07:37,776 - DEBUG - Published message to queue 'test-queue-123'.
2025-09-10 02:07:37,873 - DEBUG - Published message to queue 'test-queue-123'.
2025-09-10 02:07:37,962 - DEBUG - Published message to queue 'test-queue-123'.


In [9]:
queue_agent.get_message(queues[0])

2025-09-10 02:07:38,150 - DEBUG - Retrieved message from queue 'test-queue-123'.


{'test': 'message1'}

In [10]:
import time
time.sleep(3)
messages = {
    'ready': queue_agent.get_message_count(queues[0], message_type='ready'),
    'unacked': queue_agent.get_message_count(queues[0], message_type='unacked'),
    'total': queue_agent.get_message_count(queues[0], message_type='total'),
}
messages

{'ready': 2, 'unacked': 1, 'total': 3}

In [11]:
queue_agent.delete_queue(queues[0])

2025-09-10 02:07:49,630 - DEBUG - Deleted queue: 'test-queue-123'.


True